# Bài 1 - Cổng Toffoli

## Bối cảnh lịch sử

40 năm trước, một nhóm gồm 50 nhà tư tưởng lớn đã sắp xếp chụp một bức ảnh cùng nhau trên bãi cỏ của MIT Endicott House. Rất ít người tham dự tại hội nghị Vật Lý của Tính Toán(Physics of Computation Conference), do MIT và IBM phối hợp tổ chức, nghĩ rằng họ sẽ làm nên lịch sử vào năm 1981. Đây được cho là nơi khai sinh ra vật lý cho máy tính, và đặc biệt là lĩnh vực đang bùng nổ tính toán lượng tử, như một môn học nghiêm túc xứng đáng với sách giáo khoa hoặc khóa học ở đại học.

Chính tại hội nghị này, Feynman đã phát ngôn câu trích dẫn nổi tiếng sau này của mình: “Tự nhiên không phải là cổ điển, chết tiệt, và nếu bạn muốn mô phỏng tự nhiên, bạn nên làm nó bằng cơ học lượng tử, và thật thú vị nó là một vấn đề tuyệt vời, bởi vì nó trông không dễ dàng như vậy.”[1] Đầu tháng này, chúng tôi đã kỷ niệm 40 năm hội nghị quan trọng này. Bạn có thể đọc thêm tại [đây](https://youtu.be/GR6ANm6Z0yk).

![](resources/conference-photo.jpeg)

Một trong những chủ đề được thảo luận tại hội nghị là tính toán đảo ngược, mà Tommaso Toffoli và Edward Fredkin ở MIT đã suy nghĩ, xem xét trong những năm gần đây.[2-3] Toffoli đã đề xuất một phiên bản có thể đảo ngược của cổng AND / NAND (hiện được gọi là cổng Toffoli, hoặc cổng CCNOT). Bởi vì NAND là một cổng đa năng trong tính toán cổ điển, cổng Toffoli là một cổng logic thuận nghịch đa năng . Tính toán lượng tử là một dạng đặc biệt của tính toán đảo ngược; bất kì cổng thuận nghịch cũng có thể được thực hiện trên máy tính lượng tử, do đó cổng Toffoli cũng là cổng logic lượng tử. Tuy nhiên, cổng Toffoli không phải là cổng vạn năng cho tính toán lượng tử.. 

Trong bài tập này, chúng ta sẽ tìm hiểu về cổng Toffoli và tập cổng đa năng cho máy tính lượng tử.

### Tài liệu tham khảo
1. Feynman, Richard P. "Simulating physics with computers." Int. J. Theor. Phys 21.6/7 (1982).
1. Toffoli, Tommaso. "Reversible computing." International colloquium on automata, languages, and programming. Springer, Berlin, Heidelberg, 1980.
1. Fredkin, Edward, and Tommaso Toffoli. "Conservative logic." International Journal of theoretical physics 21.3 (1982): 219-253.

## Cổng logic cổ điển

Trong tính toán cổ điển, mô hình thường được sử dụng là logic Boolean hoặc cổng logic cổ điển. Các cổng này đại diện cho các hàm Boolean, các hàm chỉ có đầu vào và đầu ra là hệ nhị phân (0,1). Một khía cạnh thú vị của logic Boolean là tất cả các hàm nhị phân có thể được hình thành bằng cách sử dụng sự kết hợp của một số lượng nhỏ các cổng logic khác nhau. Các tập hợp này được gọi là các tập hoàn chỉnh về mặt chức năng. Trong trong những tập hợp đó, có một tập nổi tiếng đó là tập chứa cổng AND và NOT. Hai cổng này đủ để biểu diễn tất cả các hàm nhị phân. Điều này cũng đúng cho tập cổng OR và NOT. Có những tập hợp nhỏ hơn, chẳng hạn như NAND và NOR chỉ đứng một mình là đủ để tổng quát, tuy nhiên, các hàm AND, NOT và OR thường được coi là khối xây dựng của tính toán cổ điển.
<div class="alert alert-block alert-success">

**Mục tiêu**

Xây dựng một cổng Toffoli sử dụng hệ cổng cơ bản (cổng CX, RZ, SX và X) của hệ thống lượng tử IBM.

</div>

<div class="alert alert-block alert-danger">
    
Bài tập này nhằm mục đích chỉ bạn những khái niệm cơ bản của những cổng lượng tử và các xây dựng một mạch lượng tử.
1. sử dụng một cách trực quan widget Circuit Composer
1. lập trình sử dụng Qiskit. 

Nếu bạn đã quen thuộc với các cổng lượng tử và Qiskit. Bạn có thể nhảy vào thẳng <a href=#problem>bài tập</a>.
</div>

In [ ]:
# Loại bỏ những cảnh báo không cần thiết
import warnings
from matplotlib.cbook import MatplotlibDeprecationWarning
warnings.filterwarnings('ignore', category=MatplotlibDeprecationWarning)

# Nhập vào những thư viện tiêu chuẩn của Qiskit
from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Hữu ích khi có số pi
import math
pi=math.pi

## Mạch lượng tử là gì?

Mạch lượng tử là mô hình cho tính toán lượng tử trong đó tính toán là một chuỗi các cổng lượng tử. Cổng lượng tử thường biểu diễn các phép quay trên quả cầu Bloch. Chúng ta hãy xem xét một số cổng lượng tử phổ biến. 

### Cổng X 

Cổng X được biểu diễn bởi ma trân Pauli-X: 

$X = \begin{pmatrix}
0 & 1 \\
1 & 0 \\
\end{pmatrix}$

Một cổng X tương ứng với việc quay trục X của quả cầu Bloch 1 góc $\pi$ radians. Nó ánh xạ $|0\rangle$ tới $|1\rangle$ và $|1\rangle$ tới $|0\rangle$. Nó tương đương với cổng NOT trong tính toán cổ điển và đôi khi được gọi là bit-flip.

In [ ]:
x_gate=QuantumCircuit(1) # Tạo 1 mạch lượng tử với 1 qubit
x_gate.x(0)
x_gate.draw(output='mpl')

In [ ]:
backend = Aer.get_backend('statevector_simulator')
result = execute(x_gate, backend).result().get_statevector()
plot_bloch_multivector(result)

### Cổng SX

Cổng SX tương ứng với việc quay trục X của quả cầu Bloch 1 góc $\pi/2$. Nó được gọi là cổng SX để biểu thị rằng nó là căn bậc hai (square-root) của cổng X. Áp dụng cổng này 2 lần sẽ tạo ra kết quả của cổng Pauli-X. Ngược lại với cổng SX là cổng SX dagger (chuyển vị liên hợp của SX), là một phép quay $\pi/2$ theo hướng ngược lại.

$SX = \frac{1}{\sqrt{2}}\begin{pmatrix}
1+i & 1-i \\
1-i & 1+i \\
\end{pmatrix}$

In [ ]:
sx_gate = QuantumCircuit(1)
sx_gate.sx(0)  
sx_gate.draw(output='mpl')

In [ ]:
backend = Aer.get_backend('statevector_simulator')
result = execute(sx_gate, backend).result().get_statevector()
plot_bloch_multivector(result)

### Cổng RZ

Cổng RZ thực hiện một phép quay với góc $\phi$ quanh hướng của trục Z ( với $\phi$ là một số thực). Nó có ma trận như bên dưới:

$RZ = \begin{pmatrix}
1 & 0 \\
0 & e ^{i \phi } \\
\end{pmatrix}$

In [ ]:
rz_gate = QuantumCircuit(1)
rz_gate.rz(pi/2, 0)
rz_gate.draw(output='mpl')

In [ ]:
backend = Aer.get_backend('statevector_simulator')
result = execute(rz_gate, backend).result().get_statevector()
plot_bloch_multivector(result)

Bởi vì phép quay quanh trục Z, chúng ta sẽ không thể thấy được sự khác biệt khi áp dụng nó vào trạng thái mặc định $|0\rangle$, vì thể chúng ta sử dụng trạng thái được tạo ra bởi việc áp dụng cổng SX và áp dụng RZ cho nó .

In [ ]:
rz_gate.sx(0)
rz_gate.rz(pi/2, 0)
rz_gate.draw(output='mpl')

In [ ]:
backend = Aer.get_backend('statevector_simulator')
result = execute(rz_gate, backend).result().get_statevector()
plot_bloch_multivector(result)

### Cổng Hadamard
Cổng Hadamard biểu diễn phép quay góc $\pi$ quanh trục nằm giữa trục X và trục Z.
Nó ánh xạ trạng thái cơ sở $|0\rangle$ tới $\frac{|0\rangle + |1\rangle}{\sqrt{2}}$, có nghĩa là phép đo sẽ cho ra xác suất bằng nhau giữa 2 kết quả `1` hoặc `0`, tạo ra một sự 'chồng chập' các trạng thái. Trạng thái này cũng được viết là $|+\rangle$. Những gì cổng Hadamard làm là làm biến đổi giữa 2 cơ sở $|0\rangle$ $|1\rangle$ và $|+\rangle$ $|-\rangle$. 

$H = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1 \\
1 & -1 \\
\end{pmatrix}$

In [ ]:
# Áp dụng cổng H lên qubit với trạng thái |0>
h_gate = QuantumCircuit(1)
h_gate.h(0)
h_gate.draw(output='mpl')

In [ ]:
# Hãy xem kết quả
backend = Aer.get_backend('statevector_simulator')
result = execute(h_gate, backend).result().get_statevector()
plot_bloch_multivector(result)

### Cổng CX (CNOT)

Cổng điều khiển NOT (hay CNOT hay CX) hoạt động trên 2 qubits. Nó thực hiện hoạt động NOT (tương đương với việc áp dụng một cổng X) trên qubit thứ hai chỉ khi qubit đầu tiên có trạng thái $|1\rangle$ nếu không thì qubit thứ hai không thay đổi. 

Ghi chú: Qiskit đánh số cái bit của một chuỗi từ phải sang trái.

$CX = \begin{pmatrix}
1 & 0 & 0 & 0  \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
\end{pmatrix}$

In [ ]:
cx_gate = QuantumCircuit(2)
cx_gate.cx(0,1)
cx_gate.draw(output='mpl')

### Cổng CCX (Toffoli)

Cổng CCX (cổng điều khiển X kép) còn được gọi lại cổng Toffoli. Cổng CCX là một cổng ba bit, với 2 bit điều khiển và 1 bit mục tiêu giống như đầu vào và đầu ra. Nếu 2 bit điều khiển cùng ở trạng thái $|1\rangle$, nó áp dụng Pauli-X(NOT) lên qubit thứ ba. Nếu không, nó không làm gì cả. 

Ghi chú: Qiskit đánh số cái bit của một chuỗi từ phải sang trái.

$CCX = \begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
\end{pmatrix}$

In [ ]:
ccx_gate = QuantumCircuit(3)
ccx_gate.ccx(0,1,2)
ccx_gate.draw(output='mpl')

## Tạo cổng logic sử dụng cổng logic lượng tử

### Cổng NOT

Cổng NOT đảo ngược giá trị của một bit, và như đã đề cập từ trước, một cổng X có thể được xem là một cổng NOT. Bảng chân trị của một cổng X :

| Đầu vào | Đầu ra |
| --- | --- | 
| 1 | 0 |
| 0 | 1 |

In [ ]:
not_gate=QuantumCircuit(1,1) # Tạo một mạch lượng tử với 1 qubit và 1 bit cổ điển
not_gate.x(0)
not_gate.measure(0,0)
not_gate.draw(output='mpl')

### Cổng AND

Đầu ra của cổng AND chỉ đúng khi cả 2 đầu vào mang giá trị đúng. Bảng chân trị của cổng AND:

| A (Đầu vào) | B (Đầu vào) | Đầu ra |
| --- | --- | --- |
| 0 | 0 | 0 | 
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

Với cổng Toffoli, chúng ta có thể nhận được kết quả của cổng AND bằng cách diễn giải hai bit điều khiển là những bit đầu vào và bit mục tiêu là bit đầu ra. 

In [ ]:
and_gate=QuantumCircuit(3,1) # Tạo một mạch lượng tử với 3 qubit và 1 bit cổ điển
and_gate.ccx(0,1,2)
and_gate.measure(2,0)
and_gate.draw(output='mpl')

### Cổng OR

Cổng OR trả về giá trị đúng khi ít nhất một trong hai đầu vào mang giá trị đúng.
Bảng chân trị của cổng OR:

| A (Đầu vào) | B (Đầu vào) | Đầu ra |
| --- | --- | --- |
| 0 | 0 | 0 | 
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 1 |

In [ ]:
or_gate=QuantumCircuit(3,1) # Tạo một mạch lượng tử với 3 qubit và 1 bit cổ điển
or_gate.cx(1,2)
or_gate.cx(0,2)
or_gate.ccx(0,1,2)
or_gate.measure(2,0)
or_gate.draw(output='mpl')

## Sử dụng widget Circuit Composer(trình soạn mạch)

Bạn có thể quen thuộc với trình soạn mạch lượng tử của IBM, nơi bạn có thể tạo mạch trong khi sử dụng giao diện đồ họa. Với widget trình soạn mạch mới, chức năng tương tự có thể được sử dụng trong jupyter notebook. Bạn có thể tìm hiểu thêm về cách sử dụng widget tại [đây](https://quantum-computing.ibm.com/lab/docs/iql/composer-widget)

<div class="alert alert-block alert-success">

**Bài tập 1a:** Xây dựng 1 cổng NOR  (OR phủ địnhg) sử dụng trình soạn mạch
    
Bài tập này nhằm khuyến khích bạn chơi một chút với trình soạn mạch. Nó không được tính điểm. Bạn có thể bỏ qua nếu muốn. Thực hiện ô bên dưới sẽ hiển thị cho bạn một trình soạn mạch, nơi bạn có thể thêm các cổng bằng cách kéo chúng vào đúng vị trí.    
</div>

In [ ]:
from ibm_quantum_widgets import CircuitComposer
editor = CircuitComposer()
editor

### Sử dụng widget Circuit Composer để làm việc với mạch có sẵn

Chúng ta cũng có thể sử dụng trình soạn mạch để mở một mạch đã tạo trước đó. Với đoạn code dưới đây, chúng ta sẽ mở lại được mạch đã tạo ở trên, đại diện cho cổng OR. Bạn có thể sử dụng nó để kiểm tra xem bạn đã xây dựng chính xác cổng OR của mình hay chưa.

<div class="alert alert-block alert-success">

Bạn có thể chỉnh sửa mạch đã mở bằng cách kéo và thả. Cố gắng xóa phép đo ở cuối. 
    
</div>

In [ ]:
from ibm_quantum_widgets import CircuitComposer
editor2 = CircuitComposer(circuit=or_gate)
editor2

Trong ví dụ dưới đây, chúng ta lưu trữ mạch từ trước của trình soạn mạch trong biến qc2, sau đó áp dụng cổng X cho đầu ra và đo lại.

Sử dụng mạch từ việc chỉnh sửa lần trước và áp dụng cổng X cho đầu ra và đo lại. 

In [ ]:
qc2 = editor2.circuit

qc2.x(2)
qc2.measure((2), (0))

qc2.draw(output='mpl')

Bây giờ chúng ta đã tạo ra một NOR (phủ định của OR), nó nên giống với mạch mà bạn đã tự xây dựng trong bài tập đầu tiên. 

## Cổng lượng tử hỗn hợp và chi phí của chúng

Một máy tính lượng tử thực sự thường sẽ không có thực hiện vật lý của tất cả các cổng. Thay vào đó họ sử dụng một bộ nhỏ các cổng cơ bản, mà chúng tạo thành một bộ cổng đa năng, tương tự như trường hợp cổ điển, một bộ hướng dẫn có thể được sử dụng để thực hiện tất cả các hoạt động.

Vì lý do này, các mạch lượng tử phải được chuyển về các cổng cơ sở trước khi thực hiện. Điều này thường được thực hiện tự động bởi bộ chuyển tiếp(transpiler) của Qiskit khi một mạch lượng tử được gửi đến hệ thống lượng tử của IBM. Nhưng vì mục đích học tập, bạn cần phải tự mình xây dựng mạch bằng cách sử dụng các cổng cơ sở . Những cổng cơ sở của hệ thống lượng tử IBM là CX, ID, RZ, SX và X. Bạn có thể xem [hệ thống của `ibmq_mumbai`](https://quantum-computing.ibm.com/services?skip=0&systems=all&system=ibmq_mumbai) như một ví dụ.

Bây giờ chúng ta hãy nhìn vào mạch bên dưới:

In [ ]:
qc = QuantumCircuit(2)
qc.sxdg(0)
qc.t(1)
qc.draw(output='mpl')

Bây giờ chúng ta hãy xem sự phân tách của mạch trên có thể trông như thế nào đối với một máy tính lượng tử, chỉ sử dụng các cổng cơ sở. 

In [ ]:
qc = QuantumCircuit(2)
qc.sx(0)
qc.sx(0)
qc.sx(0)
qc.rz(pi/4,1)
qc.draw(output='mpl')

Như bạn có thể thấy, chúng ta hiện chỉ sử dụng các cổng cơ sở, nhưng vì lý do này, nhiều cổng được sử dụng hơn. Mạch càng có nhiều cổng thì hoạt động của nó càng phức tạp. Vì vậy, khi chúng ta muốn tính toán chi phí của một mạch, chúng ta sẽ xem xét số lượng cổng được sử dụng. Tuy nhiên, không phải tất cả các cổng đều được coi là bằng nhau về mặt chi phí, vì vậy khi chúng ta tính toán chi phí của một mạch, chúng ta sử dụng công thức sau: 

$$
Cost = 10 N_{CNOT} + N_{other}
$$

trong đó $N_{CNOT}$ là số lượng cổng CNOT và $N_{other}$ là số lượng các cổng còn lại.

### Cổng Hadamard

Như đã nói ở trên, tất cả các hoạt động có thể được thể hiện chỉ bằng cách sử dụng các cổng cơ bản. Như một ví dụ, chúng ta sẽ chỉ ra cách xây dựng cổng Hadamard bằng cách sử dụng bộ cổng cơ sở của mình. Chúng ta không có cổng cơ sở thực hiện phép quay trực tiếp quanh trục nằm giữa trục X và trục Z, vì vậy thay vào đó chúng ta sử dụng các phép quay quanh trục X và trục Z để đạt được kết quả tương tự.

Bạn có đoán được chúng ta cần thực hiện những phép quay nào không? 

In [ ]:
q=QuantumRegister(1)
c=ClassicalRegister(1)
qc=QuantumCircuit(q,c)
qc.rz(pi/2, 0)
qc.sx(0)
qc.rz(pi/2, 0)
qc.draw(output='mpl')

Như bạn có thể nhớ, đây là mạch chúng ta đã có ở trên, khi chúng ta hình dung sự quay của cổng RZ. Chúng ta có thể thấy ở trên cổng RZ đầu tiền không làm gì cả, khi chúng ta ở trạng thái $|0\rangle$ hoặc $|1\rangle$. Vì vậy nó có vẻ không cần thiết. Tuy nhiên, nếu chúng ta ở trạng thái $|+\rangle$ và $|-\rangle$ phép quay đầu tiên có tác dụng. Chúng ta có kịch bản ngược lại, bởi vì sau khi áp dụng cổng SX chúng ta quay trở lại trạng thái $|0\rangle$ và $|1\rangle$ và cổng RZ thứ hai không có tác dụng gì cả.

### Sự quay có điều khiển

Chúng ta đã thấy ở trên cổng điều khiển NOT, giờ hãy xem qua một ví dụ về cách tạo một phép quay có điểu khiển quanh trục Y. Phép quay $\theta$ có thể là bất kì phép quay nào, không nhất thiết phải là $\pi$, đây là một ví dụ. 

In [ ]:
qc = QuantumCircuit(2)
theta = pi # Theta có thể là bất cứ gì (pi chỉ là 1 lựa chọn ngẫu nhiên)
qc.ry(theta/2,1)
qc.cx(0,1)
qc.ry(-theta/2,1)
qc.cx(0,1)
qc.draw(output='mpl')

Khi đi qua mạch này, chúng ta có thể thấy rằng nếu qubit đầu tiên là 0 thì hai phép quay sẽ triệt tiêu lẫn nhau và không có gì xảy ra. 

Mặt khác, nếu qubit đầu tiên là 1, chúng ta sẽ thu được một trạng thái bằng với việc áp dụng của phép quay $\theta / 2$ hai lần ,tạo thành phép quay ban đầu của chúng ta $\theta$. Điều này hoạt động vì trục X và Y là trực giao.
<div class="alert alert-block alert-danger">
Để quay quanh các trục khác, bạn có thể cần sử dụng một vài thủ thuật khác. 
</div>

### Sự quay có kiểm soát kép

Ở trên chúng ta đã thấy một ví dụ về việc thực hiện một phép quay có điểu khiển quanh trục $Y$.
Bây giờ, giả sử chúng ta có phép quay có điểu khiển (quanh trục mà ta muốn) và muốn xây dựng từ đó một phép quay có điều khiển kép, điều mà chỉ áp dụng nếu hai qubit điều khiển là 1 tương tự như cổng CCX. 

In [ ]:
qc = QuantumCircuit(3)
theta = pi # Theta có thể là bất cứ gì (pi chỉ là 1 lựa chọn ngẫu nhiên)
qc.cp(theta/2,1,2)
qc.cx(0,1)
qc.cp(-theta/2,1,2)
qc.cx(0,1)
qc.cp(theta/2,0,2)
qc.draw()

Trong mạch này nếu cả qubit thứ nhất và qubit thứ hai cùng là 0, sẽ không có chuyện gì xảy ra. Nếu chỉ có qubit thứ hai là 1, đầu tiên ta áp dụng phép quay $\pi/2$ và sau đó một phép quay $-\pi/2$ sẽ triệt tiêu lẫn nhau. Nếu chỉ có qubit thứ nhất là 1, thì qubit thứ hai sẽ trở thành 1 sau cổng CX đầu tiên nên một phép quay $-\pi/2$ sẽ được áp dụng và sau đó một phép quay $\pi/2$ sẽ lại được áp dụng và hai phép quay này sẽ lại triệt tiêu lẫn nhau.

Nếu cả quibt thứ nhất và qubit thứ hai cùng là 1 thì qubit thứ nhất sẽ được áp dụng một phép quay $\pi/2$ sau đó qubit thứ hai sẽ trở thành 0 nên phép quay tiếp theo sẽ không được áp dụng và sau đó nó sẽ được trả về 1. Sau đó một phép quay $\pi/2$ khác sẽ được áp dụng bởi vì qubit thứ nhất là 1. Do đó chúng ta có hai lần quay $\pi/2$ mà cùng nhau tạo thành một phép quay $\pi$.

## Bài tập

<div id='problem'></div>
<div class="alert alert-block alert-success">

Chúng ta đã thấy cách xây cổng Hadamard với bộ cổng cơ sở của chúng ta ở trên và bây giờ chúng ta cũng muốn xây cổng Toffoli. Tại sao lại là cổng Toffoli? Như đã đề cập ở trên, cổng Toffoli cũng là một cổng đa năng cho tính toán cổ điển giống như cổng NAND, nhưng nó có thể đảo ngược. Ngoài ra, nó còn xây dựng một tập cổng vạn năng đơn giản cho tính toán lượng tử nếu được kết hợp với cổng Hadamard. 

Chúng ta đã thấy một số ví dụ về cách thể hiện những cổng phức tạp bằng cách sử dụng những cổng cơ bản, bây giờ chúng tôi muốn sử dụng kiến thức đã học được để xây dựng một cổng Toffoli chỉ sử dụng những cổng cơ bản của chúng ta. Để giải bài tập này, các ví dụ trên về cách xây dựng và sử dụng phép quay có điều khiển sẽ rất hữu ích. Thách thức lớn nhất là xây dựng các phép quay có kiểm soát cần thiết.
    
Bạn có thể sử dụng code ở dưới để sử dụng widget trình soạn để tạo nên mạch lượng tử của mình.
    
</div>


<div class="alert alert-block alert-danger">

Xin nhắc lại rằng, các cổng cơ sở của hệ thống lượng tử IBM là cổng CX, RZ, SX và X, vì vậy các cổng khác không được cho phép.

Tất nhiên, chúng ta cũng muốn cố gắng giảm thiểu chi phí . 
    
$$
Cost = 10 N_{CNOT} + N_{other}
$$
    
</div>


In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute
from ibm_quantum_widgets import CircuitComposer
editorEx = CircuitComposer() 
editorEx
##### Xây dựng mạch lượng tử của bạn ở đây sử dụng widget trình soạn cổng

In [ ]:
# Bạn cũng có thể lập trình mạch lượng tử của mình bằng code Qiskit 

circuit = QuantumCircuit(3)

# Viết code của bạn ở giữa những dòng này - Bắt đầu





# Viết code của bạn ở giữa những dòng này - Kết thúc

In [ ]:
# Thực hiện mạch sử dụng qasm_simulator
qc = editorEx.circuit 
#qc = circuit # Bỏ comment dòng này nếu bạn muốn nộp mạch đã xây dựng sử dụng code Qiskit

qc.draw(output='mpl')

In [ ]:
# Kiểm tra đáp án của bạn sử dụng code sau
from qc_grader import grade_ex1
grade_ex1(qc)

In [ ]:
# Nộp kết quả của bạn. Bạn có thể nộp lại kết quả bất kì lúc nào.
from qc_grader import submit_ex1
submit_ex1(qc)

## Thông tin thêm

**Tạo bởi:** Marcel Pfaffhauser, Brahmani Thota, Junye Huang

**Dịch bởi:** Bao Bach

**Phiên bản:** 1.0.1